In [1]:
# from https://github.com/cran/GSA/blob/master/R/GSA.read.gmt.R
GSA.read.gmt = function(filename){
    # Read in and parse a gmt file (gene set file) from the  Broad institute
    # this is tricky, because each lines (geneset) has a variable length
    #  I read the file twice, first to pick up the geneset name and description
    # in the   first two  columns, then I read it all in as a long string

    # The beginning and end of each gene set in the string
    # is determined by matching
    # BOTH on  geneset name and description (since geneset names sometimes
    # occur as genenames elsewhere in the file)

    a = scan(filename,what = list("",""),sep = "\t", quote = NULL, fill = T, flush = T,multi.line = F)
    geneset.names = a[1][[1]]

    geneset.descriptions = a[2][[1]]

    dd = scan(filename,what = "",sep = "\t", quote = NULL)


    nn = length(geneset.names)
    n = length(dd)
    ox = rep(NA,nn)

    ii = 1
    for(i in 1:nn){
        while((dd[ii] != geneset.names[i]) | (dd[ii+1] != geneset.descriptions[i]) ){
            ii = ii+1
        }
        ox[i] = ii
        ii = ii+1
    }

    genesets = vector("list",nn)

    for(i in 1:(nn-1)){
        i1 = ox[i]+2
        i2 = ox[i+1]-1
        geneset.descriptions[i] = dd[ox[i]+1]
        genesets[[i]] = dd[i1:i2]
    }

    geneset.descriptions[nn] = dd[ox[nn]+1]
    genesets[[nn]] = dd[(ox[nn]+2):n]
    out = list(genesets = genesets,geneset.names = geneset.names, geneset.descriptions = geneset.descriptions)
    class(out) = "GSA.genesets"
    return(out)
}

In [13]:
source("~/wisdom/r/data_analysis_environment.R")
library(igraph)

proteome <- read_tsv("../data/mutations/proteome_information.txt")

id2symbol <- proteome %>%
    select(GeneId,Symbol) %>%
    unique

drivers <- read_tsv("../data/intogen_cancer_drivers-2014.12b/Mutational_drivers_per_tumor_type.tsv",comment="#") %>%
  set_colnames(c("Symbol","Tumor")) %>%
  .$Symbol

network.PPI <- read_tsv("../data/eporta/raw_tables/interactions_found_more_than_three_times.txt", col_names = F) %>%
    select(-X3) %>%
    set_colnames(c("Gene1","Gene2"))

network.affectedPPI <- read_tsv("../results/supplementary_files/supplementary_file_4.tsv") %>%
    filter(Feature_type == "Pfam" & Effect_on_interaction %in% c('Loss','Gain')) %>%
    select(GeneId,GeneId_partner) %>%
    unique %>%
    graph.data.frame(directed = FALSE)

complexes.raw <- read_delim("../data/genesets/allComplexes.csv", delim = ";") %>%
    set_colnames(c("id","name","synonym","organism","uniprots","geneids","method",
                   "pmid","FunCat.categories","functional.comment","disease.comment","subunit.comment")) %>%
    filter(organism == "Human") %>% 
    select(name,geneids)

complexes <- complexes.raw %>%
    apply(1, function(x) strsplit(x[2], ",") %>% unlist) %>%
    set_names(complexes.raw$name)

canonical.patways.raw <- GSA.read.gmt("../data/genesets/c2.cp.v4.0.entrez.gmt")

canonical.patways <- canonical.patways.raw$genesets
names(canonical.patways) <- canonical.patways.raw$geneset.names
        
mrna.raw <- read_tsv("../data/genesets/akerman.csv")

mrna <- apply(mrna.raw, 1, function(x){
            genes <- strsplit(x[2], ",") %>% unlist %>% sub("^\\s+", "", .)
            id2symbol %>%
                filter(Symbol %in% genes) %>% 
                .$GeneId
        } ) %>% set_names(mrna.raw$Category)
        
genesets <- c(complexes, canonical.patways, mrna)

Warning message:
: 2895 parsing failures.
row col   expected     actual
  1  -- 13 columns 12 columns
  2  -- 13 columns 12 columns
  3  -- 13 columns 12 columns
  4  -- 13 columns 12 columns
  5  -- 13 columns 12 columns
... ... .......... ..........
.See problems(...) for more details.

In [24]:
# calculate modules
modules <- cluster_louvain(network.affectedPPI)
module.names <- paste0("Module_", 1:length(modules))
module.genes <- lapply(groups(modules), function(x){ 
    id2symbol %>% filter(GeneId %in% x) %>% .$Symbol %>% paste(collapse=",")
}) %>% unlist
module.drivers  <- lapply(groups(modules), function(x) {
    id2symbol %>% filter(GeneId %in% x) %>% .$Symbol %>% intersect(drivers) %>% length
}) %>% unlist

# number of coincidences between modules and genesets
k <- lapply(groups(modules), function(m){
    lapply(genesets, intersect, m) %>% lapply(length) %>% as.numeric
}) %>% do.call("c",.)

# size of the genesets
n <- lapply(genesets, length) %>% as.numeric
# number of genes in any of the modules
m <- groups(modules) %>% unlist %>% unique %>% length
# total number of expressed genes
N <- proteome %>% .$GeneId %>% unique %>% length

# binomial test
## for each module and geneset, we draw n genes (size of the geneset), and observe a
## subset k were PPIs are affected. Is this higher than the share of the genome covered
## by the modules?
p <- mapply(function(k,n,p){binom.test(k, n, p, alternative = "greater")$p.value}, k, n, m/N)

module.test <- data.frame(Module = rep(module.names, each=length(genesets)), 
                          Module_components = rep(module.genes, each=length(genesets)), 
                          Geneset = names(genesets), Geneset_size = n, p = p, 
                          Intersection = k, Number_drivers = rep(module.drivers, each=length(genesets))) %>%
    mutate( padj = p.adjust(p)) %>%
    arrange(p)

write_tsv(module.test, "../results/networks/annotated_modules.tsv")

In [23]:
module.drivers  <- lapply(groups(modules), function(x) {
    id2symbol %>% filter(GeneId %in% x) %>% .$Symbol %>% intersect(drivers) %>% length
}) %>% unlist
head(module.drivers)

1 2 3 4 5 6 
1 0 0 2 0 0

In [4]:
# modules with a geneset assigned
module.test %>%
    filter(padj < 0.05) %>%
    select(Module) %>%
    unique %>%
    nrow
# total number of modules
length(modules)

[1] 8

[1] 197

In [5]:
# number of genesets
sum(module.test$padj < 0.05)

[1] 37

In [6]:
# size of the significant genesets
module.test %>%
    filter(padj < 0.05) %>%
    select(Geneset,Geneset_size) %>%
    unique %>%
    .$Geneset_size %>% 
    summary

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  10.00   15.00   34.00   52.03   77.50  161.00 